In [1]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


DATA DESCRIPTION

You are provided with historical sales data for 45 Walmart stores located in different regions. Each store contains a number of departments, and you are tasked with predicting the department-wide sales for each store.

In addition, Walmart runs several promotional markdown events throughout the year. These markdowns precede prominent holidays, the four largest of which are the Super Bowl, Labor Day, Thanksgiving, and Christmas. The weeks including these holidays are weighted five times higher in the evaluation than non-holiday weeks. Part of the challenge presented by this competition is modeling the effects of markdowns on these holiday weeks in the absence of complete/ideal historical data.

stores.csv

This file contains anonymized information about the 45 stores, indicating the type and size of store.

train.csv

This is the historical training data, which covers to 2010-02-05 to 2012-11-01. Within this file you will find the following fields:

    Store - the store number
    Dept - the department number
    Date - the week
    Weekly_Sales -  sales for the given department in the given store
    IsHoliday - whether the week is a special holiday week

features.csv

This file contains additional data related to the store, department, and regional activity for the given dates. It contains the following fields:

    Store - the store number
    Date - the week
    Temperature - average temperature in the region
    Fuel_Price - cost of fuel in the region
    MarkDown1-5 - anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available 
    after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA.
    CPI - the consumer price index
    Unemployment - the unemployment rate
    IsHoliday - whether the week is a special holiday week
    
For convenience, the four holidays fall within the following weeks in the dataset (not all holidays are in the data):

    Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
    Labor Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
    Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
    Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

###Stores Data

In [65]:
stores = pd.read_csv('stores.csv')#.dropna()
stores[:10]

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875
5,6,A,202505
6,7,B,70713
7,8,A,155078
8,9,B,125833
9,10,B,126512


In [66]:
#there are store types A,B,C; total of 45 stores
#about half of stores are of type A

stores['Type'].value_counts()

A    22
B    17
C     6
Name: Type, dtype: int64

###Train Data

In [208]:
train = pd.read_csv('train.csv').dropna()

del train['Dept']
print(len(train))

421570


In [69]:
print len(train) #is 421,570

421570


In [188]:
#there are 45 different stores being evaluated --> can look at this doing .value_counts().count()
#train['Store'].value_counts()
train_groupby_store_dept = train.groupby(['Store', 'Dept'])[['Weekly_Sales']].mean()
print(len(train_groupby_store_dept))

3331


In [82]:
#average weekly sales for each store and order to see which one has highest mean sales
train_groupby_store = train.groupby(['Store'])[['Weekly_Sales']].mean()
train_groupby_store[:45].sort_values(by = 'Weekly_Sales', ascending = False)[:10]

,Weekly_Sales
Store,
20,29508.301592
4,29161.210415
14,28784.851727
13,27355.136891
2,26898.070031
10,26332.303819
27,24826.984536
6,21913.243624
1,21710.543621


In [193]:
train_groupby_store_date = train.groupby(['Store', 'Date'])[['Weekly_Sales']].sum()
train_groupby_store_date[:10]

Weekly_Sales
Store Date                  
1     1/13/12     1459601.17
      1/14/11     1391013.96
      1/20/12     1394393.84
      1/21/11     1327405.42
      1/27/12     1319325.59
      1/28/11     1316899.31
      1/6/12      1550369.92
      1/7/11      1444732.28
      10/1/10     1453329.50
      10/12/12    1573072.81

###Features Data

In [170]:
features = pd.read_csv('features.csv')
features['index'] = features['']
features['Datetime'] = pd.to_datetime(features['Date'], errors = 'raise')
mask = (features['Datetime'] <= datetime.date(2012,10,27))

#features[:10]

#2069
print len(features[mask])

6435


In [187]:
#code for merging two different dataframes
#df_movie_ratings = ratings.merge(movies, left_on='movie_id', right_on='movie_id')
#df = df_movie_ratings.merge(users, left_on='user_id', right_on='user_id')

df_stores_features = stores.merge(features, left_on='Store', right_on='Store')

df_stores_features['UniqueID'] = df_stores_features.Store.apply(str)+" "+df_stores_features.Date.apply(str)
df_stores_features[:10]
df = df_stores_features.merge(train, left_on='UniqueID', right_on='UniqueID1', suffixes=['_stores_features', '_train'])
print(len(df))
#df
#df_groupby = df.groupby(['Store_stores_features', 'Date'])[['Weekly_Sales']].sum()
#df_groupby

#print len(df_groupby)  #8190

#df_stores_train = stores.merge(train, left_on='Store', right_on='Store')
#df = df_stores_train.merge(features, left_on = 'Date', right_on = 'Date')
#df[50:100]

#df[:10]

421570


In [109]:
#df_groupby_stores = df.groupby('Store')[['Weekly_Sales']].max()
#df_groupby_stores.sort_values(by='Weekly_Sales', ascending=False)[:5]

,Weekly_Sales
Store,
10,693099.36
35,649770.18
14,474330.10
20,422306.25
27,420586.57
